In [69]:
from gurobipy import *
import pandas as pd
import numpy as np
import grblogtools as glt
from scipy.spatial.distance import squareform, pdist
import datetime
import math

In [97]:
df = pd.read_csv('data\instance05.txt', header=None)
n_S = int(df_main['n_S'])
n_C = int(df_main['n_C'])
n_L = int(df_main['n_L'])
n_K = int(df_main['n_K'])
n_D = int(df_main['n_D'])
B = int(df_main['B'])

df_main = df.iloc[:2,:]
df_main.columns = df_main.iloc[0]
df_main = df_main[1:]

df_car =  df.iloc[3:n_C+4,:]
df_car.columns = df_car.iloc[0]
df_car = df_car.dropna(axis=1)
df_car = df_car[1:]

df_car_lv = df.iloc[n_C+5:n_C+n_L+6,:]
df_car_lv.columns = df_car_lv.iloc[0]
df_car_lv = df_car_lv.dropna(axis=1)
df_car_lv = df_car_lv[1:]
print(df_car_lv)

df_order = df.iloc[n_C+n_L+7:n_C+n_L+n_K+8,:]
df_order.columns = df_order.iloc[0]
df_order = df_order.dropna(axis=1)
df_order = df_order[1:]
df_order['Time span'] = df_order.apply(lambda row: pd.to_datetime(row['Return time']) - pd.to_datetime(row['Pick-up time']), axis=1)
df_order['Time units'] = df_order['Time span'].apply(lambda x: math.ceil(x.total_seconds() / 1800))
first_period_start = pd.to_datetime('2023/01/01 00:00')
time_unit = pd.Timedelta('30 minutes')
df_order['Pick-up time (ordinal)'] = df_order.apply(lambda row: pd.to_datetime(row['Pick-up time']) - first_period_start, axis=1).apply(lambda x: math.ceil(x.total_seconds() / 1800))
df_order['Return time (ordinal)'] = df_order.apply(lambda row: pd.to_datetime(row['Return time']) - first_period_start, axis=1).apply(lambda x: math.ceil(x.total_seconds() / 1800))
print(df_order)

df_distance = df.iloc[n_C+n_L+n_K+9:n_C+n_L+n_K+n_S**2+10,:]
df_distance.columns = df_distance.iloc[0]
df_distance = df_distance.dropna(axis=1)
df_distance = df_distance[1:]
distance = df_distance.pivot(index='From', columns='To', values='Distance')
distance = distance.apply(pd.to_numeric, errors='coerce') // 30

print(distance)

11 Car level Hour rate
12         1       100
13         2       400
15 Order ID Level Pick-up station Return station      Pick-up time  \
16        1     1               1              6  2023/01/04 01:00   
17        2     1               4              5  2023/01/01 16:00   
18        3     1               5              1  2023/01/02 03:30   
19        4     1               1              2  2023/01/03 12:00   
20        5     2               6              6  2023/01/01 00:00   
21        6     1               3              4  2023/01/04 23:00   
22        7     2               2              1  2023/01/02 03:30   
23        8     2               6              6  2023/01/05 04:30   
24        9     2               3              4  2023/01/01 19:30   
25       10     1               4              2  2023/01/01 06:00   

15       Return time       Time span  Time units  Pick-up time (ordinal)  \
16  2023/01/04 20:00 0 days 19:00:00          38                     146   
17  2023

In [ ]:
eg1 = Model("eg1")

#-------- Add variables as a list ---------#
# xmk = 1 if order k is processed on car m
x = []
for m in n_C:
    x.append([])
    for k in n_K:
        x[i].append(eg1.addVar(lb = 0, vtype = GRB.BINARY, name = "x" + str(m+1) + str(k+1)))

# Om = latest completion time of orders on car m
O = []
for m in n_C:
    O.append(eg1.addVar(lb=0, vtype = INTEGER, name = "O" + str(m+1)))

B = eg1.addVar(lb = 0, vtype = GRB.INTEGER, name = "max_rearrangement time")


In [ ]:
eg1.setObjective(
    quicksum(R[k] for k in n_K)\
    ,GRB.MAXIMIZE
)